In [1]:
import time 
start = time.time()

import os
# os.environ["pRT_input_data_path"] = "/home/mvasist/pRT/input_data"

import numpy as np
import pylab as plt
import matplotlib.ticker as mticker

import pymultinest

plt.rcParams['figure.figsize'] = (10, 6)
from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc 
from petitRADTRANS.retrieval.parameter import Parameter
from petitRADTRANS.retrieval.models import emission_model_diseq

from sbi.inference import SNRE_A, SNRE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi.types import Array, OneOrMore, ScalarFloat

import pandas as pd
import sys
import csv
import h5py
import json
from pathlib import Path
import math
import glob
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision
from torchvision import datasets, transforms, models
from torchsummary import summary



In [2]:
pwd

'/home/mvasist/scripts_new'

In [3]:
print(f"Torch: {torch.__version__}")
print(f"TorchVision: {torchvision.__version__}")

Torch: 1.9.1
TorchVision: 0.10.1


In [4]:
torch.cuda.is_available() == True

False

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [6]:
# path_to_file = "/home/mvasist/scripts_new/datasets/13params_traintest_vit.h5"

# with h5py.File(path_to_file, 'r') as hdf:
#     data = np.array(hdf.get('data')).astype(np.float32)
#     label = np.array(hdf.get('labels')).astype(np.float32)

# hdf.close()

# print(np.shape(data)) #[params,spectra]
# data = np.pad(data, ((0,0),(0,0),(0,2)), 'constant')
# print(np.shape(data))  #962
# print(np.shape(data[:,:,13:])) #949

# # Normalize
# a_min = np.min(data[:,:,13:])  #axis=0
# a_max = np.max(data[:,:,13:])
# spec_norm = (data[:,:,13:]-a_min)/(a_max-a_min)
# print(np.shape(spec_norm))
# data_norm = np.c_[data[:,:,:13], spec_norm]
# print(np.shape(data_norm))

# #Tensor
# target = torch.from_numpy(label)
# inpt = torch.from_numpy(data_norm)
# inpt_ = inpt.unsqueeze_(1)
# inpt_ch = transforms.Lambda(lambda x: x.repeat(1, 3, 1, 1))(inpt)

In [7]:
# for ig in glob.iglob('/home/mvasist/scripts_new/datasets/dataset/_/onehot/*.h5'):
#     print(ig)

files = glob.glob('/home/mvasist/scripts_new/datasets/dataset/_/onehot/*.h5')
len(files)


46667

In [8]:
class HDF5Dataset(data.Dataset):
    """Represents an abstract HDF5 dataset.
    
    Input params:
        file_path: Path to the folder containing the dataset (one or multiple HDF5 files).
        load_data: If True, loads all the data immediately into RAM. Use this if
            the dataset is fits into memory. Otherwise, leave this at false and 
            the data will load lazily.
        data_cache_size: Number of HDF5 files that can be cached in the cache (default=3).
    """
    def __init__(self, file_path, load_data, data_cache_size=2):
        super().__init__()
        self.data_info = []
        self.data_cache = {}
        self.data_cache_size = data_cache_size

        # Search for all h5 files
        p = Path(file_path)
        #print(p)
        assert(p.is_dir())
        
        files = sorted(p.glob('13params_vit__1_*.h5'))     ###################################################
        if len(files) < 1:
            raise RuntimeError('No hdf5 datasets found')

        for h5dataset_fp in files:
            self._add_data_infos(str(h5dataset_fp.resolve()), load_data)
            
    def __getitem__(self, index):
        # get data
        x = self.get_data("data", index) #cache data
        x = torch.from_numpy(x)
        x = x.unsqueeze_(1)
        #x = transforms.Lambda(lambda h: h.repeat(1, 3, 1, 1))(x) #adding 3 channels 
        x = torch.nn.functional.pad(x, (0, 2, 0, 0)) #padding - 962 div by 13
        #x = x.view(-1,3,1,962)
        #print('get_item: ', x.size())

        # get label
        y = self.get_data("label", index) 
        y = torch.from_numpy(y)
        #y = y.view(-1,1)
        #print('len: ', y.size())
        return (x, y)

    def __len__(self):
        return len(self.get_data_infos('data'))
    
    def _add_data_infos(self, file_path, load_data):
        with h5py.File(file_path, 'r') as h5_file:
            # Walk through all datasets, extracting them
            for dname, ds in h5_file.items():
                #print(dname, ds)
                # if data is not loaded its cache index is -1
                idx = -1
                if load_data:
                    # add data to the data cache
                    idx = self._add_to_cache(h5_file[dname][()], file_path)

                # type is derived from the name of the dataset; we expect the dataset
                # name to have a name such as 'data' or 'label' to identify its type
                # we also store the shape of the data in case we need it
                self.data_info.append({'file_path': file_path, 'type': dname, 'shape': h5_file[dname][()].shape, 'cache_idx': idx})

    def _load_data(self, file_path): #into cache
        """Load data to the cache given the file
        path and update the cache index in the
        data_info structure.
        """
        with h5py.File(file_path,'r') as h5_file:
            for dname, ds in h5_file.items():
                #print(dname)
                idx = self._add_to_cache(h5_file[dname][()], file_path) #0 for data, 1 for labels

                # find the beginning index of the hdf5 file we are looking for
                file_idx = next(i for i,v in enumerate(self.data_info) if v['file_path'] == file_path)

                # the data info should have the same index since we loaded it in the same way
                self.data_info[file_idx + idx]['cache_idx'] = idx
                

        # remove an element from data cache if size was exceeded
        if len(self.data_cache) > self.data_cache_size:
            # remove one item from the cache at random
            removal_keys = list(self.data_cache)
            removal_keys.remove(file_path)
            self.data_cache.pop(removal_keys[0])
            # remove invalid cache_idx
            self.data_info = [{'file_path': di['file_path'], 'type': di['type'], 'shape': di['shape'], 'cache_idx': -1} if di['file_path'] == removal_keys[0] else di for di in self.data_info]

    def _add_to_cache(self, data, file_path):
        """Adds data to the cache and returns its index. There is one cache
        list for every file_path, containing all datasets in that file.
        """
        if file_path not in self.data_cache:
            self.data_cache[file_path] = [data]
        else:
            self.data_cache[file_path].append(data)
            
        return len(self.data_cache[file_path]) - 1

    def get_data_infos(self, type):
        """Get data infos belonging to a certain type of data.
        """
        
        data_info_type = [di for di in self.data_info if di['type'] == type]
        return data_info_type
        

    def get_data(self, type, i):
        # This method loads the data in the file that the spectrum is in
        
        """Call this function anytime you want to access a chunk of data from the
            dataset. This will make sure that the data is loaded in case it is
            not part of the data cache.
        """
        #size of dataset in each file
        
        fp = self.get_data_infos(type)[i]['file_path']
        if fp not in self.data_cache:
            self._load_data(fp)
        
        # get new cache_idx assigned by _load_data_info
        cache_idx = self.get_data_infos(type)[i]['cache_idx']
        return self.data_cache[fp][cache_idx]



In [9]:
dataset = HDF5Dataset('/home/mvasist/scripts_new/datasets/dataset/_/onehot/', load_data=False, data_cache_size=4)
# print(len(dataset))

split = [0.9, 0.1]
split_train = '0.9'
batch_size = 4 
indices = list(range(len(dataset)))
s = int(np.floor(split[1] * len(dataset)))

In [10]:
#shuffling
np.random.seed(111)
np.random.shuffle(indices)
train_indices, val_indices = indices[s:], indices[:s]
#print(train_indices, val_indices)
train_sampler, val_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=2, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=2, sampler=val_sampler)

In [11]:
img = dataset.__getitem__(2)
print(np.shape(img[0]))
print(np.shape(img[1]))

torch.Size([200, 1, 1, 962])
torch.Size([200, 2])


In [12]:
# print(np.shape(train_dataloader.dataset))


In [13]:
for ind, s in enumerate(train_dataloader):
#     print(ind)
#     print(s[0].size(), s[1].size())
#     s_0 = s[0].view(-1,1,1,962)
#     print(s_0.size())
    print(np.shape(s[1]))
    break
    
# print('')    

# for ind, s in enumerate(val_dataloader):
#     print(ind,np.shape(s[0]), np.shape(s[1]))
    

torch.Size([4, 200, 2])


In [14]:
dataloaders = {}
dataloaders['train'], dataloaders['val'] = train_dataloader, val_dataloader


In [15]:
len(dataloaders['train']), len(dataloaders['train'].dataset), len(dataloaders['val'])

(12, 49, 1)

In [16]:
def binary_acc(y_pred, y_test):
    #print(y_pred.argmax(axis=1), y_test.argmax(axis=1))
    correct_results_sum = (y_pred.argmax(axis=1) == y_test.argmax(axis=1)).sum().float()
    #print('crs', y_pred.argmax(axis=1) == y_test.argmax(axis=1), correct_results_sum, len(y_test))
    acc = correct_results_sum/len(y_test)
    acc = torch.round(acc * 100)
    
    return acc

In [17]:
dataset_size = len(dataset)
print(dataset_size)

49


In [18]:
from vit_pytorch.efficient import ViT
from linformer import Linformer
from vit_pytorch import ViT as ViT_modified

#962 = 13 * 74  - div into 74 patches 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #torch.device("cpu") #

model = ViT_modified(n_classes = 1,
                    image_size = (1, 962),  # image size is a tuple of (height, width)
                    patch_size = (1, 13),    # patch size is a tuple of (height, width)
                    dim = 16,
                    depth = 3,
                    heads = 16,
                    mlp_dim = 512,
                    dropout = 0.1,
                    emb_dropout = 0.1
                ).to(device)

summary(model, (1, 1, 962))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 73, 16]              --
|    └─Rearrange: 2-1                    [-1, 73, 13]              --
|    └─Linear: 2-2                       [-1, 73, 16]              224
├─Dropout: 1-2                           [-1, 74, 16]              --
├─Transformer: 1-3                       [-1, 74, 16]              --
├─Identity: 1-4                          [-1, 16]                  --
├─Sequential: 1-5                        [-1, 1]                   --
|    └─LayerNorm: 2-3                    [-1, 29]                  58
|    └─Linear: 2-4                       [-1, 1]                   30
Total params: 312
Trainable params: 312
Non-trainable params: 0
Total mult-adds (M): 0.49
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 73, 16]              --
|    └─Rearrange: 2-1                    [-1, 73, 13]              --
|    └─Linear: 2-2                       [-1, 73, 16]              224
├─Dropout: 1-2                           [-1, 74, 16]              --
├─Transformer: 1-3                       [-1, 74, 16]              --
├─Identity: 1-4                          [-1, 16]                  --
├─Sequential: 1-5                        [-1, 1]                   --
|    └─LayerNorm: 2-3                    [-1, 29]                  58
|    └─Linear: 2-4                       [-1, 1]                   30
Total params: 312
Trainable params: 312
Non-trainable params: 0
Total mult-adds (M): 0.49
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

#print(n.element_size() * n.nelement() /1e6, 'MB')


In [20]:
#train

def train(n_epochs, model):
    
    best_loss = 0.0
    for epoch in range(n_epochs):
        gc.collect()
        torch.cuda.empty_cache()
        for phase in ['train', 'val']:
            if phase == 'train':
                s = 0
                model.train()  # Set model to training mode
            else:
                s = 1
                model.eval()   # Set model to evaluate mode
                        
            #both    
            running_loss = 0.0
            a = 0 
            for batch_idx, sample in enumerate(dataloaders[phase]):
                inputs = sample[0].view(-1,1,1,962).to(device)
                target = sample[1][0].view(-1,1).long().to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    output = model(inputs) #[None, ...]
                    loss = criterion(output, target) #torch.squeeze() .argmax(axis=1)
                    #acc = binary_acc(torch.squeeze(output), torch.squeeze(target))
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   
                
                    running_loss += 1 * loss.item() * inputs.size(0) #loss for the phase/whole dataset
                
                if batch_idx % 1 == 0: 
                    a+= len(sample[0])
                    print('{} epoch: {} [{}/{} ({:0.0f}%)]\tLoss: {:.6f}\tAcc: {:.2f}'.format(phase,epoch,\
                            a,int(np.ceil(len(dataset)*split[s])),np.floor((100.*a)/(len(dataset)*split[s])), loss.item(), acc))
                                
            if phase == 'train':
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[0]))
            else:
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[1]))

            if phase == 'val': 
                if epoch ==  (n_epochs-1) or running_loss < best_loss:
                    print('saving')
                    best_loss = running_loss
                    model_path = os.path.join(model_dir, 'model_vit.pth')
                    torch.save(model.state_dict(), model_path)
                    
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=4)
                    
#         print('--------------------------------------------------------------------')
        

In [ ]:
nb_epoch = 1
criterion = nn.BCEWithLogitsLoss() #nn.BCELoss() nn.CrossEntropyLoss()
# summary(model, (1, 1, 960))

optimizer = optim.Adam(model.parameters(), lr=float(0.001))

model_dir = '/home/mvasist/scripts_new/model/'
metrics_path = os.path.join(model_dir, 'metrics__vit.json')

metrics = {
    'model': model_dir,
    'optimizer': optimizer.__class__.__name__,
    'criterion': criterion.__class__.__name__,
#     'scheduler': scheduler.__class__.__name__,
    'dataset_size': int(len(dataset)),
    'train_size': int(split[0]*len(dataset)),
    'test_size': int(split[1]*len(dataset)),
    'n_epoch': nb_epoch,
    'batch_size': batch_size,
#     'learning_rate': [],
    'train_loss': [],
    'val_loss': []
}

train(nb_epoch, model)

In [ ]:
# Testing
test_dataset_size= 1000
# Define name of the hdf5 file containing the test data:
path_to_datasets = '/home/mvasist/scripts_new/model/datasets/'
test_file = path_to_datasets+"13params_traintest_bce_trial.h5"

transfo = transforms.Compose([Normalize(), ToTensor(), Resize(256), CenterCrop(224)])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# Create the dataset object:xc
test_dataset = spectrum_dataset(path_to_file = test_file, 
                          size = test_dataset_size,
                          transform = transfo)

In [ ]:
prediction_list = []
ratio_list = []
p= []
acc=0

for id in range(test_dataset_size):
    sample = test_dataset[id]
    lteratio = sample['target'].numpy()
    spectrum = sample['input'].unsqueeze(0)
    prediction = model_loop(spectrum).detach().numpy()[0]

    prediction_list.append(prediction)
    ratio_list.append(lteratio)

    if (prediction > 0.5): 
        p.append(np.float32([1]))
    else: 
        p.append(np.float32([0]))

    if p[id] == lteratio: 
        acc+=1

acc = (acc/test_dataset_size)*100

In [ ]:
acc,id

In [ ]:
posterior = inference.build_posterior(density_estimator)

In [ ]:
observation = torch.load('observations/6param_observation_TintLkIRLgLH2OLCH4LCO.pt') 

In [ ]:

start = time.time()
sampls= 1000

samples = posterior.sample((sampls,), x=observation)
log_probability = posterior.log_prob(samples, x= observation)
end= time.time()
print('it takes: '+ str((end-start)/3600) + ' hrs')

#save samples

In [25]:
target = torch.ones([10, 64], dtype=torch.float32)  # 64 classes, batch size = 10
output = torch.full([10, 64], 1.5)  # A prediction (logit)
pos_weight = torch.ones([64])  # All weights are equal to 1
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion(output, target)

tensor(0.2014)

In [27]:
output.size()

torch.Size([10, 64])

In [28]:
target

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.,